# 1. load data

In [1]:
import cv2
import os
from tqdm import *
import numpy 


data_root = 'data/'
label_list = []
label_to_image_path_list = {}
label_to_int = {}
int_to_label = {}

# load meta data 
cnt = 0
for f1 in os.listdir(data_root):
    label = f1
    label_to_int[label] = cnt
    int_to_label[cnt] = label
    cnt += 1
    label_list.append(label)
    f2 = os.path.join(data_root, f1)
    label_to_image_path_list[label] = []
    for f3 in os.listdir(f2):
        f4 = os.path.join(f2, f3)
        label_to_image_path_list[label].append(f4)

print( 'label_to_int={0}'.format(label_to_int) )
print( 'int_to_label={0}'.format(int_to_label) )


label_to_int={'c1': 0, 'c2': 1, 'c3': 2}
int_to_label={0: 'c1', 1: 'c2', 2: 'c3'}


In [2]:
# load image to color images
label_to_image_arr_list = {}
for label in label_to_image_path_list:
    label_to_image_arr_list[label] = []
    image_path_list = label_to_image_path_list[label]
    for image_path in tqdm(image_path_list):
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        label_to_image_arr_list[label].append(img)

# print load status
for label in label_to_image_arr_list:
    print('{0}: loaded {1} images, shape={2}'.format(label, len(label_to_image_arr_list[label]), 
                                                    label_to_image_arr_list[label][0].shape))

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

c1: loaded 5 images, shape=(50, 50, 3)
c2: loaded 5 images, shape=(50, 50, 3)
c3: loaded 5 images, shape=(50, 50, 3)


In [3]:
# images to x_data, y_data
    
x_data = []
y_data = []
for label in label_to_image_arr_list:
    for img in label_to_image_arr_list[label]:
        x_data.append( img )
        y_data.append( label_to_int[label] )

x_data = numpy.array(x_data)
y_data = numpy.array(y_data)
x_data = x_data.reshape( (len(x_data), 3, 50, 50) )
print( 'x_data.shape={0}, y_data.shape={1}'.format(x_data.shape, y_data.shape) )


x_data.shape=(15, 3, 50, 50), y_data.shape=(15,)


# 2. model and train

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.models import resnet18
import torchvision
import numpy as np

In [5]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    print('device is GPU')
else:
    print('device is CPU')

device is CPU


In [6]:
# create the model
model = resnet18()
model.fc = nn.Linear(512, 3)

In [7]:
costfunc = nn.MultiMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # Adam as optimizer

In [8]:
# data loader
from sklearn.model_selection import train_test_split

# train test split
x_train, x_test, y_train, y_test = train_test_split( x_data, y_data, 
                                                    test_size=0.1, random_state=123 )
# output 
print('x_train={0}, x_test={1}, y_train={2}, y_test={3}'.format(
    x_train.shape, x_test.shape, y_train.shape, y_test.shape
) )

x_train=(13, 3, 50, 50), x_test=(2, 3, 50, 50), y_train=(13,), y_test=(2,)


In [9]:
# data to tensor
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [10]:
x_train.shape

torch.Size([13, 3, 50, 50])

In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
for epoch in range(10):
    # Train
    model.train()
    # predict
    y_pred = model(x_train)
    # calculate loss
    loss = costfunc(y_pred, y_train)
    # gradient decent
    optimizer.zero_grad()
    loss.backward()
    # weights update
    optimizer.step()
    
    # Validation
    model.eval()
    y_test_pred = model(x_test)
    loss_test = costfunc(y_test_pred, y_test)
    print(loss_test)
    yhat = y_test_pred.detach().numpy()
    # round to class values
    yhat = yhat.round()
    yhat = np.argmax(yhat, axis=1)

tensor(36984.4414, grad_fn=<MultiMarginLossBackward0>)
tensor(681435., grad_fn=<MultiMarginLossBackward0>)
tensor(373883.7812, grad_fn=<MultiMarginLossBackward0>)
tensor(250862.9531, grad_fn=<MultiMarginLossBackward0>)
tensor(184401.0625, grad_fn=<MultiMarginLossBackward0>)
tensor(116544.8359, grad_fn=<MultiMarginLossBackward0>)
tensor(71547.8125, grad_fn=<MultiMarginLossBackward0>)
tensor(39440.7539, grad_fn=<MultiMarginLossBackward0>)
tensor(22168.2910, grad_fn=<MultiMarginLossBackward0>)
tensor(13848.6396, grad_fn=<MultiMarginLossBackward0>)


In [18]:
yhat, y_test

(array([2, 2], dtype=int64), tensor([1, 2]))